In [1]:
import pandas as pd
import logging
import numpy as np
import os
import re
from scipy.stats import zscore

from dataWraggler import *

In [2]:
Normalization()

In [3]:
fileName = 'cleaned_2.csv'
df = pd.read_csv(fileName)
df

,surface,draw_size,tourney_level,tourney_date,player_id,player_name,player_hand,player_ht,player_age,opponent_id,...,score,round,player_rank,player_rank_points,opponent_rank,opponent_rank_points,set_score_diff,set_score_sum,game_score_diff,game_score_sum
0,0,5.0,2,20160104,105062,Mikhail Kukushkin,2.0,-0.625925,1.281134e-01,104797,...,6-2 7-5,5,0.015385,-5.040373e-01,0.016393,-3.005089e-01,2,2,6,20
1,0,5.0,2,20160104,104797,Denis Istomin,2.0,0.200149,3.881309e-01,105062,...,6-2 7-5,5,0.016393,-3.005089e-01,0.015385,-5.040373e-01,-2,2,-6,20
2,0,5.0,2,20160104,103285,Radek Stepanek,2.0,-0.342321,2.100869e+00,105583,...,6-0 6-3,5,0.005076,-7.388598e-01,0.013158,-3.826132e-01,2,2,9,15
3,0,5.0,2,20160104,105583,Dusan Lajovic,2.0,-0.972702,-4.277399e-01,103285,...,6-0 6-3,5,0.013158,-3.826132e-01,0.005076,-7.388598e-01,-2,2,-9,15
4,0,5.0,2,20160104,106071,Bernard Tomic,2.0,0.792093,-9.124610e-01,103917,...,6-4 6-3,5,0.055556,-8.365906e-02,0.014085,-3.571051e-01,2,2,5,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37327,0,2.0,1,20220304,105278,Alexis Klegou,3.0,0.000000,1.182531e+00,138846,...,1-6 6-4 4-1 RET,4,0.000532,-9.222702e-01,0.000944,-8.502854e-01,-1,3,0,22
37328,0,2.0,1,20220304,209409,Coleman Wong,2.0,0.000000,-2.104786e+00,200583,...,6-4 6-4,4,0.013583,-1.046911e-16,0.009230,-1.812464e-16,2,2,4,20
37329,0,2.0,1,20220304,200583,Delmas Ntcha,2.0,0.000000,-1.243611e+00,209409,...,6-4 6-4,4,0.009230,-1.812464e-16,0.013583,-1.046911e-16,-2,2,-4,20
37330,0,2.0,1,20220304,208414,Sylvestre Monnou,2.0,0.000000,-1.779606e+00,207132,...,6-4 6-4,4,0.013583,-1.046911e-16,0.009230,-1.812464e-16,2,2,4,20


In [4]:
df_train = df.where(df['tourney_date'] < 20210000)
df_test = df.where(df['tourney_date'] >= 20210000)

df_train = df_train.dropna()
df_test = df_test.dropna()

In [6]:
df_train.dtypes

surface                 float64
draw_size               float64
tourney_level           float64
tourney_date            float64
player_id               float64
player_name              object
player_hand             float64
player_ht               float64
player_age              float64
opponent_id             float64
opponent_name            object
opponent_hand           float64
opponent_ht             float64
opponent_age            float64
score                    object
round                   float64
player_rank             float64
player_rank_points      float64
opponent_rank           float64
opponent_rank_points    float64
set_score_diff          float64
set_score_sum           float64
game_score_diff         float64
game_score_sum          float64
dtype: object

In [7]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [8]:
X_train = df_train.drop(columns=['player_name', 'opponent_name', 'score', 'set_score_diff', 'set_score_sum', 'game_score_diff', 'game_score_sum'])
y_train = df_train[['set_score_diff', 'set_score_sum', 'game_score_diff', 'game_score_sum']]
X_test = df_test.drop(columns=['player_name', 'opponent_name', 'score', 'set_score_diff', 'set_score_sum', 'game_score_diff', 'game_score_sum'])
y_test = df_test[['set_score_diff', 'set_score_sum', 'game_score_diff', 'game_score_sum']]

In [9]:
model = DecisionTreeRegressor()
model.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [10]:
y_pred = model.predict(X_test)

In [30]:
set_score_diff_test = y_test['set_score_diff'].to_numpy()
set_score_diff_pred = y_pred[:,0]

set_score_diff_mse = mean_squared_error(set_score_diff_test, set_score_diff_pred)
print(f'set_score_diff\nMSE: {set_score_diff_mse:.2f}')

l = len(set_score_diff_test)
succ = 0
no = 0
for i in range(l):
    if set_score_diff_pred[i] == 0:
        no += 1
    if set_score_diff_test[i] * set_score_diff_pred[i] > 0:
        succ += 1
print(l, succ, no)

set_score_diff
MSE: 3.14
11298 8693 250
